<a href="https://colab.research.google.com/github/zhrldnpftl/DKU-Capstone-DeepPhish/blob/main/Voice-context-Detection/0.%EA%B8%88%EC%9C%B5%EA%B0%90%EB%8F%85%EC%9B%90_%EC%9D%8C%EC%84%B1%ED%8C%8C%EC%9D%BC_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [금융감독원(FSS) 웹사이트](https://www.fss.or.kr/fss/bbs/B0000203/list.do?menuNo=200686&bbsId=&cl1Cd=&pageIndex=10&sdate=&edate=&searchCnd=1&searchWrd=)에서 보이스피싱 사례 mp4 파일 자동 다운로드

# 0. 필요한 라이브러리 설치

In [ ]:
#!pip install beautifulsoup4 requests moviepy

# 1. 디렉토리 생성

In [ ]:
import os

# 최상위 폴더
base_dir = "/content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data"

# mp4 파일 저장용 폴더
mp4_dir = os.path.join(base_dir, "mp4")

# mp4 변환된 wav 파일 저장용 폴더
wav_dir = os.path.join(base_dir, "wav")

# 폴더 생성
os.makedirs(base_dir, exist_ok=True)
os.makedirs(mp4_dir, exist_ok=True)
os.makedirs(wav_dir, exist_ok=True)

# 2. 웹사이트에서 mp4 링크 가져오기

In [ ]:
import re                                     # 정규표현식 사용을 위한 모듈
import requests                               # 웹 요청을 위한 모듈
from bs4 import BeautifulSoup                 # HTML 파싱을 위한 모듈
from urllib.parse import urljoin, unquote     # 상대 URL을 절대 URL로 바꿔주는 함수

# 기본 url 설정
base_url   = "https://www.fss.or.kr"

# 첫 번째 페이지 URL (전체 페이지 수를 가져오기 위해 필요)
first_page_url = "https://www.fss.or.kr/fss/bbs/B0000203/list.do?menuNo=200686&pageIndex=1"

# 첫 페이지 요청
res = requests.get(first_page_url)
soup = BeautifulSoup(res.text, "html.parser")

## <금융감독원 "바로 이 목소리" 음성 추출>

[1] 전체 페이지 수 가져오기

In [ ]:
# 전체 페이지 수가 들어 있는 div 선택
count_total_div = soup.select_one("div.count-total")

# 해당 div의 전체 텍스트 가져오기
text = count_total_div.get_text()

# 정규표현식으로 "/12페이지"에서 숫자 12만 추출
match = re.search(r"/\s*(\d+)\s*페이지", text)

# 추출된 숫자를 정수형으로 변환하여 total_pages에 저장 (매칭 안 되면 기본값 1)
total_pages = int(match.group(1)) if match else 1

print(f"📄 전체 페이지 수: {total_pages}")

📄 전체 페이지 수: 12


[2] 1페이지 ~ 전체 페이지 순환하며 상세 페이지 수집

In [ ]:
# 페이지별 url 템플릿
page_template = "https://www.fss.or.kr/fss/bbs/B0000203/list.do?menuNo=200686&pageIndex={}"

# 상세 페이지 url들을 저장할 리스트
all_detail_urls = []

# 1페이지 ~ 전체 페이지 수까지 순환하면서 상세 링크 수집
for page in range(1, total_pages + 1):
  url   = page_template.format(page)
  res   = requests.get(url)
  soup  = BeautifulSoup(res.text, "html.parser")

  # 게시글 링크들을 선택하여 수집
  for a_tag in soup.select("div.bd-list-thumb-a ul li a[href]"):
    href      = a_tag["href"]             # 상대 경로
    full_url  = urljoin(base_url, href)   # 절대 url로 변환
    all_detail_urls.append(full_url)

# 수집된 상세페이지 링크 개수 출력
print(f"🔎 수집된 상세페이지 수: {len(all_detail_urls)}")

🔎 수집된 상세페이지 수: 94


In [ ]:
all_detail_urls

['https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130785&menuNo=200686&pageIndex=1',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130784&menuNo=200686&pageIndex=1',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130783&menuNo=200686&pageIndex=1',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130781&menuNo=200686&pageIndex=1',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130780&menuNo=200686&pageIndex=1',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130778&menuNo=200686&pageIndex=1',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130768&menuNo=200686&pageIndex=1',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130767&menuNo=200686&pageIndex=1',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130766&menuNo=200686&pageIndex=2',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130764&menuNo=200686&pageIndex=2',
 'https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130762&menuNo=200686&pageIndex=2',
 'https://

[3] 상세 페이지에서 정보 추출

In [ ]:
# 저장 경로 생성
download_dir = "/content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp4"
os.makedirs(download_dir, exist_ok=True)

# 수집된 정보를 저장할 리스트
collected_data = []

In [ ]:
# 요소 추출 테스트용 상세페이지
detail_url = all_detail_urls[0]
print("detail_url >>" , detail_url)

# 상세 페이지 요청
res = requests.get(detail_url)
soup = BeautifulSoup(res.text, "html.parser")

# 1. 제목 추출
name_tag = soup.select_one('.name')
print("name_tag >> ", name_tag)

if name_tag:
  file_text = name_tag.get_text(strip=True)
  print("file_text >> ", file_text)

  # 가장 앞쪽 기준 .mp4 기준으로 잘라내기
  split_index = file_text.find('.mp4')
  if split_index != -1:
    filename = file_text[:split_index]
    print("filename >> ", filename)
  else:
    print(".mp4를 찾을 수 없습니다.")

detail_url >> https://www.fss.or.kr/fss/bbs/B0000203/view.do?nttId=130785&menuNo=200686&pageIndex=1
name_tag >>  <span class="name">금감원_보이스피싱_12_수정07.mp4.mp4<span>
								(파일크기: 18,748KB)</span></span>
file_text >>  금감원_보이스피싱_12_수정07.mp4.mp4(파일크기: 18,748KB)
filename >>  금감원_보이스피싱_12_수정07


In [ ]:
# 2. mp4 다운로드 링크 추출
file_list_tag = soup.select_one('.file-list')
print("file_list_tag >> ", file_list_tag)

if file_list_tag:
  a_tag = file_list_tag.select_one('a')
  print("a_tag >> ", a_tag)

  if a_tag and a_tag.has_attr('href'):
    relative_url = a_tag['href'].replace('&amp;', '&')
    download_url = "https://www.fss.or.kr" + relative_url
    print("download_url >> ", download_url)

    # 3. 파일 다운로드
    response = requests.get(download_url, stream=True)
    file_path = os.path.join(download_dir, filename)

    with open(file_path, 'wb') as f:
      for chunk in response.iter_content(chunk_size=1024*1024):   # 1MB씩 저장
        if chunk:
          f.write(chunk)
    print("파일 다운로드 완료:", file_path)
  else:
    print("다운로드 링크가 없습니다.")
else:
  print("파일 리스트 태그를 찾을 수 없습니다.")

file_list_tag >>  <dl class="file-list">
<dt>영상파일</dt>
<dd>
<a href="/fss/cmmn/vodApi/apiVodDownload.do?mediaId=a93358fef9464e1eb7c83f9a7c1517fc&amp;atchFileId=85bdcbf96ebf469fbf71cf34b16fe51b&amp;fileSn=1">
<span class="file-name">
<i class="ico-mov"></i>
<span class="name">금감원_보이스피싱_12_수정07.mp4.mp4<span>
								(파일크기: 18,748KB)</span></span>
</span>
</a>
</dd>
</dl>
a_tag >>  <a href="/fss/cmmn/vodApi/apiVodDownload.do?mediaId=a93358fef9464e1eb7c83f9a7c1517fc&amp;atchFileId=85bdcbf96ebf469fbf71cf34b16fe51b&amp;fileSn=1">
<span class="file-name">
<i class="ico-mov"></i>
<span class="name">금감원_보이스피싱_12_수정07.mp4.mp4<span>
								(파일크기: 18,748KB)</span></span>
</span>
</a>
download_url >>  https://www.fss.or.kr/fss/cmmn/vodApi/apiVodDownload.do?mediaId=a93358fef9464e1eb7c83f9a7c1517fc&atchFileId=85bdcbf96ebf469fbf71cf34b16fe51b&fileSn=1
파일 다운로드 완료: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp4/금감원_보이스피싱_12_수정07


In [ ]:
# 4. 날짜 추출
inline_tag = soup.select_one('.inline')
print("inline_tag >> ", inline_tag)

reg_date = None

if inline_tag:
  dt_tags = inline_tag.select('dt')
  dd_tags = inline_tag.select('dd')

  for dt, dd in zip(dt_tags, dd_tags):
    if dt.get_text(strip=True) == "등록일":
      reg_date = dd.get_text(strip=True)
      break
if reg_date:
  print("등록일 : ", reg_date)
else:
  print("등록일을 찾을 수 없습니다.")

inline_tag >>  <dl class="inline">
<dt>등록일</dt>
<dd>2023-10-06</dd>
<dt>조회수</dt>
<dd>
			13547
		</dd>
</dl>
등록일 :  2023-10-06


In [ ]:
# 5. 중요 내용 추출
dbdata_tag = soup.select_one('.dbdata')
# print("dbdata >> ", dbdata)

important_lines = []

if dbdata_tag:
  # dbdata 안의 모든 <p> 태그 추출
  p_tags = dbdata_tag.find_all('p')

  for p in p_tags:
    text = p.get_text(strip=True)
    # 텍스트가 비어있지 않고, 시간 형식이 포함된 경우만 필터링
    if text and re.search(r'\d{2}:\d{2}', text):
      important_lines.append(text)

print("중요 내용 추출 결과 >>")
for line in important_lines:
  print("-", line)

중요 내용 추출 결과 >>
- 00:07~00:15 *주된수법 : 고립된 장소로 유도하여 주변인의 간섭이나 도움을 차단
- 00:39~01:00 *주된수법 : 구체적인 사건 내용과 전문용어 구사
- 01:44~01:55 *모범대응 : 내 명의 계좌는 어플에서 지금 확인이 가능한데 왜 조회가 되지 않냐고 반문
- 02:47~02:54 *주된수법 : 피해자 입증을 하지 않으면 출석 소환장을 발부하겠다고 압박
- 03:03~ : 바로 이 '목소리'입니다. 의심하GO, 끊GO, 확인하GO, 3GO 언제나 실천하세요! - 금융감독원 -


[4] 전체 페이지 순회하며 모든 페이지의 mp4 파일 다운로드

In [ ]:
# 전체 상세 페이지 순회
for idx, detail_url in enumerate(all_detail_urls):
    print(f"\n[{idx+1}/{len(all_detail_urls)}] Processing: {detail_url}")

    try:
        # 상세 페이지 요청
        res = requests.get(detail_url)
        soup = BeautifulSoup(res.text, "html.parser")

        # 1. 제목 추출
        name_tag = soup.select_one('.name')
        if not name_tag:
            print("제목(.name) 태그 없음. 스킵.")
            continue

        file_text = name_tag.get_text(strip=True)
        split_index = file_text.find('.mp4')
        if split_index == -1:
            print(".mp4 확장자 없음. 스킵.")
            continue

        filename = file_text[:split_index + 4]  # .mp4까지 포함
        safe_filename = filename.replace('/', '_')  # 파일명에 슬래시 있을 경우 대비
        file_path = os.path.join(download_dir, safe_filename)

        # 2. mp4 다운로드 링크 추출
        file_list_tag = soup.select_one('.file-list')
        if not file_list_tag:
            print("파일 리스트 태그(.file-list) 없음. 스킵.")
            continue

        a_tag = file_list_tag.select_one('a')
        if not a_tag or not a_tag.has_attr('href'):
            print("다운로드 a 태그 없음. 스킵.")
            continue

        relative_url = a_tag['href'].replace('&amp;', '&')
        download_url = "https://www.fss.or.kr" + relative_url

        # 3. 파일 다운로드
        response = requests.get(download_url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024*1024):  # 1MB
                    if chunk:
                        f.write(chunk)
            print("✅ 다운로드 완료:", file_path)

            collected_data.append({
                "url": detail_url,
                "filename": safe_filename,
                "path": file_path
            })
        else:
            print(f"❌ 다운로드 실패. HTTP 상태코드: {response.status_code}")
    except Exception as e:
        print(f"❌ 예외 발생: {e}")

print("\n📦 모든 다운로드 작업 완료.")

In [ ]:
# 시작 인덱스 설정 (31번 인덱스 = 32번째 항목)
start_index = 31  # 🔧 수정됨

for idx, detail_url in enumerate(all_detail_urls[start_index:], start=start_index):
    print(f"\n[{idx+1}/{len(all_detail_urls)}] Processing: {detail_url}")

    try:
        # 상세 페이지 요청
        res = requests.get(detail_url)
        soup = BeautifulSoup(res.text, "html.parser")

        # 1. 제목 추출
        name_tag = soup.select_one('.name')
        if not name_tag:
            print("제목(.name) 태그 없음. 스킵.")
            continue

        file_text = name_tag.get_text(strip=True)
        split_index = file_text.find('.mp4')
        if split_index == -1:
            print(".mp4 확장자 없음. 스킵.")
            continue

        filename = file_text[:split_index + 4]  # .mp4까지 포함
        safe_filename = filename.replace('/', '_')  # 파일명에 슬래시 있을 경우 대비
        file_path = os.path.join(download_dir, safe_filename)

        # 2. mp4 다운로드 링크 추출
        file_list_tag = soup.select_one('.file-list')
        if not file_list_tag:
            print("파일 리스트 태그(.file-list) 없음. 스킵.")
            continue

        # 🔧 기존 방식 + 대체 방식 통합
        a_tag = soup.select_one('.file-list a')
        if not a_tag or not a_tag.has_attr('href'):
            video_tag = soup.select_one('video[src]')
            if not video_tag:
                print("video 태그 없음. 스킵.")
                continue
            relative_url = video_tag['src'].replace('&amp;', '&')
        else:
            relative_url = a_tag['href'].replace('&amp;', '&')

        download_url = "https://www.fss.or.kr" + relative_url

        # 🔧 중복 제거: relative_url 재할당 제거
        # relative_url = a_tag['href'].replace('&amp;', '&')
        # download_url = "https://www.fss.or.kr" + relative_url

        # 3. 다운로드 요청 (Referer, User-Agent 헤더 포함) 🔧 추가됨
        headers = {
            "Referer": detail_url,
            "User-Agent": "Mozilla/5.0"
        }

        response = requests.get(download_url, headers=headers, stream=True, timeout=30)  # 🔧 timeout 추가
        if response.status_code == 200:
            total = 0  # 🔧 다운로드 진행 상태 출력 추가
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024*1024):  # 1MB
                    if chunk:
                        f.write(chunk)
                        total += len(chunk)
                        print(f"  ⏬ 다운로드 중... {total / (1024*1024):.2f} MB")  # 🔧 진행상황 출력
            print("✅ 다운로드 완료:", file_path)

            collected_data.append({
                "url": detail_url,
                "filename": safe_filename,
                "path": file_path
            })
        else:
            print(f"❌ 다운로드 실패. HTTP 상태코드: {response.status_code}")
    except Exception as e:
        print(f"❌ 예외 발생: {e}")

print("\n📦 모든 다운로드 작업 완료.")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  ⏬ 다운로드 중... 38.35 MB
  ⏬ 다운로드 중... 38.36 MB
  ⏬ 다운로드 중... 38.36 MB
  ⏬ 다운로드 중... 38.36 MB
  ⏬ 다운로드 중... 38.36 MB
  ⏬ 다운로드 중... 38.36 MB
  ⏬ 다운로드 중... 38.37 MB
  ⏬ 다운로드 중... 38.37 MB
  ⏬ 다운로드 중... 38.37 MB
  ⏬ 다운로드 중... 38.37 MB
  ⏬ 다운로드 중... 38.38 MB
  ⏬ 다운로드 중... 38.38 MB
  ⏬ 다운로드 중... 38.38 MB
  ⏬ 다운로드 중... 38.39 MB
  ⏬ 다운로드 중... 38.39 MB
  ⏬ 다운로드 중... 38.39 MB
  ⏬ 다운로드 중... 38.40 MB
  ⏬ 다운로드 중... 38.40 MB
  ⏬ 다운로드 중... 38.40 MB
  ⏬ 다운로드 중... 38.41 MB
  ⏬ 다운로드 중... 38.41 MB
  ⏬ 다운로드 중... 38.41 MB
  ⏬ 다운로드 중... 38.42 MB
  ⏬ 다운로드 중... 38.42 MB
  ⏬ 다운로드 중... 38.42 MB
  ⏬ 다운로드 중... 38.42 MB
  ⏬ 다운로드 중... 38.43 MB
  ⏬ 다운로드 중... 38.43 MB
  ⏬ 다운로드 중... 38.43 MB
  ⏬ 다운로드 중... 38.43 MB
  ⏬ 다운로드 중... 38.43 MB
  ⏬ 다운로드 중... 38.44 MB
  ⏬ 다운로드 중... 38.44 MB
  ⏬ 다운로드 중... 38.44 MB
  ⏬ 다운로드 중... 38.45 MB
  ⏬ 다운로드 중... 38.45 MB
  ⏬ 다운로드 중... 38.45 MB
  ⏬ 다운로드 중... 38.46 MB
  ⏬ 다운로드 중... 38.46 MB
  ⏬ 다운로드 중... 38.46 MB
  ⏬ 다운로드 중... 38.47 MB
  ⏬ 다운로드 중... 38.47 M

#<금융감독원 '그놈 목소리' 대출사기형 크롤링>

In [ ]:
import re                                     # 정규표현식 사용을 위한 모듈
import requests                               # 웹 요청을 위한 모듈
from bs4 import BeautifulSoup                 # HTML 파싱을 위한 모듈
from urllib.parse import urljoin, unquote     # 상대 URL을 절대 URL로 바꿔주는 함수

# 기본 url 설정
base_url   = "https://www.fss.or.kr"

# 첫 번째 페이지 URL (전체 페이지 수를 가져오기 위해 필요)
first_page_url_2 = "https://www.fss.or.kr/fss/bbs/B0000206/list.do?menuNo=200690"

# 첫 페이지 요청
res = requests.get(first_page_url_2)
soup = BeautifulSoup(res.text, "html.parser")

[1] 전체 페이지 수 수집

In [ ]:
# 전체 페이지 수가 들어 있는 div 선택
count_total_div = soup.select_one("div.count-total")

# 해당 div의 전체 텍스트 가져오기
text = count_total_div.get_text(strip=True)

# 정규표현식으로 "/19페이지"에서 숫자 19만 추출
match = re.search(r"/\s*(\d+)\s*페이지", text)

# 추출된 숫자를 정수형으로 변환하여 total_pages에 저장 (매칭 안 되면 기본값 1)
total_pages_2 = int(match.group(1)) if match else 1

print(f"📄 전체 페이지 수: {total_pages_2}")

📄 전체 페이지 수: 19


[2] 1페이지 ~ 전체 페이지 순환하며 상세 페이지 수집

In [ ]:
# 페이지별 url 템플릿
page_template_2 = "https://www.fss.or.kr/fss/bbs/B0000206/list.do?menuNo=200690&pageIndex={}"

# 상세 페이지 URL들을 저장할 리스트
all_detail_urls_2 = []

# 각 페이지를 순회하며 링크 추출
for page in range(1, total_pages_2 + 1):
    url = page_template_2.format(page)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    # 각 게시글 행을 순회
    for tr in soup.select("tbody tr"):
        num_td = tr.select_one("td.num")
        print("num_td >> ", num_td)
        title_a = tr.select_one("td.title a")
        print("titlte_a >> ", title_a)

        if num_td and title_a:
            post_number = num_td.get_text(strip=True)
            print("post_number >> ", post_number)
            href = title_a["href"]
            print("href >> ", href)
            full_url = urljoin(base_url, href)
            print("full_url >> ", full_url)

            all_detail_urls_2.append((post_number, full_url))

# 수집된 상세페이지 수 출력
print(f"🔎 수집된 상세페이지 수: {len(all_detail_urls_2)}")

# 예시 출력 (앞부분만)
for num, url in all_detail_urls_2[:5]:
    print(f"📌 글번호 {num} → {url}")

num_td >>  <td class="num"> 
					185
					</td>
titlte_a >>  <a href="/fss/bbs/B0000206/view.do?nttId=36744&amp;menuNo=200690&amp;pageIndex=1">KB저축은행입니다. 핸드폰 어떤거 사용하세요?</a>
post_number >>  185
href >>  /fss/bbs/B0000206/view.do?nttId=36744&menuNo=200690&pageIndex=1
full_url >>  https://www.fss.or.kr/fss/bbs/B0000206/view.do?nttId=36744&menuNo=200690&pageIndex=1
num_td >>  <td class="num"> 
					184
					</td>
titlte_a >>  <a href="/fss/bbs/B0000206/view.do?nttId=36743&amp;menuNo=200690&amp;pageIndex=1">우리은행 직원인데요 영업점에선 절 모를수도있어요</a>
post_number >>  184
href >>  /fss/bbs/B0000206/view.do?nttId=36743&menuNo=200690&pageIndex=1
full_url >>  https://www.fss.or.kr/fss/bbs/B0000206/view.do?nttId=36743&menuNo=200690&pageIndex=1
num_td >>  <td class="num"> 
					183
					</td>
titlte_a >>  <a href="/fss/bbs/B0000206/view.do?nttId=36742&amp;menuNo=200690&amp;pageIndex=1">KB저축은행인데요 카카오톡친구추천부탁드립니다</a>
post_number >>  183
href >>  /fss/bbs/B0000206/view.do?nttId=36742&menuNo=200690&pageIndex=1
ful

[3] 전체 페이지 순회하며 모든 페이지의 mp3 파일 다운로드

In [ ]:
import os

# 삭제할 디렉토리 경로
download_dir_2 = "/content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형"

# 디렉토리 내 파일 삭제
deleted_count = 0
for filename in os.listdir(download_dir_2):
    file_path = os.path.join(download_dir_2, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)
        deleted_count += 1
        print(f"🗑️ 삭제됨: {file_path}")

print(f"\n✅ 총 {deleted_count}개 파일 삭제 완료.")


🗑️ 삭제됨: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/185_KB저축은행입니다. 핸드폰 어떤거 사용하세요_.mp3
🗑️ 삭제됨: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/184_우리은행 직원인데요 영업점에선 절 모를수도있어요.mp3
🗑️ 삭제됨: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/183_KB저축은행인데요 카카오톡친구추천부탁드립니다.mp3
🗑️ 삭제됨: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/182_영업점에선 절 모를수도있어요(...___).mp3
🗑️ 삭제됨: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/181_OO캐피탈입니다. 심사결과 확인되어서 연락드렸습니다..mp3
🗑️ 삭제됨: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/180_OO저축은행입니다..mp3
🗑️ 삭제됨: /content/dri

In [ ]:
import os

# 저장 경로 생성
download_dir_2 = "/content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형"
os.makedirs(download_dir_2, exist_ok=True)

# 다운로드 결과를 저장할 리스트
collected_mp3_data = []

In [ ]:
# 파일명에서 특수문자 제거 및 길이 제한
def sanitize_filename(title, max_length=100):
    title = re.sub(r'[\\/*?:"<>|]', "_", title)  # 파일명에 쓸 수 없는 문자 제거
    return title.strip()[:max_length]

# 전체 상세 페이지 순회
for idx, (post_number, detail_url) in enumerate(all_detail_urls_2):
    print(f"\n[{idx+1}/{len(all_detail_urls_2)}] Processing: {detail_url}")

    try:
        # 상세 페이지 요청
        res = requests.get(detail_url)
        soup = BeautifulSoup(res.text, "html.parser")

        # 제목 추출
        title_tag = soup.select_one("h2.subject")
        if not title_tag:
            print("❌ 제목 태그 없음. 스킵.")
            continue
        title_text = title_tag.get_text(strip=True)
        safe_title = sanitize_filename(title_text)
        print("title >> ", safe_title)

        # ✅ 등록일 추출 (dt, dd 쌍을 순회하면서 "등록일" 텍스트를 가진 dt를 찾음)
        post_date = "unknown"
        bd_tag = soup.select_one("div.bd-view")
        if bd_tag:
            dd_tags = bd_tag.select("dd")
            dd_tag  = dd_tags[1]
            post_date = dd_tag.get_text().strip()

        print("등록일 >>", post_date)


        # video 태그 선택
        video_tag = soup.select_one("div.dbdata video")
        if not video_tag or not video_tag.has_attr("src"):
            print("⚠️ video 태그 또는 src 없음. 스킵.")
            continue

        # src에서 상대경로 추출
        relative_url = video_tag["src"].replace("&amp;", "&")
        download_url = urljoin("https://www.fss.or.kr", relative_url)

        # 파일명 생성 (글번호_제목 형식)
        filename = f"{post_number}.mp3"  # 글 번호만 사용
        file_path = os.path.join(download_dir_2, filename)

        # 파일 다운로드
        response = requests.get(download_url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024*1024):  # 1MB 단위
                    if chunk:
                        f.write(chunk)
            print(f"✅ 다운로드 완료: {file_path}")

            collected_mp3_data.append({
                "post_number": post_number,
                "title": title_text,
                "date": post_date,
                "url": detail_url,
                "filename": filename,
                "path": file_path
            })
        else:
            print(f"❌ 다운로드 실패. 상태코드: {response.status_code}")

    except Exception as e:
        print(f"❌ 예외 발생: {e}")

print("\n📦 모든 mp3 다운로드 작업 완료.")


[1/185] Processing: https://www.fss.or.kr/fss/bbs/B0000206/view.do?nttId=36744&menuNo=200690&pageIndex=1
title >>  KB저축은행입니다. 핸드폰 어떤거 사용하세요_
등록일 >> 2020-12-10
✅ 다운로드 완료: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/185.mp3

[2/185] Processing: https://www.fss.or.kr/fss/bbs/B0000206/view.do?nttId=36743&menuNo=200690&pageIndex=1
title >>  우리은행 직원인데요 영업점에선 절 모를수도있어요
등록일 >> 2020-12-10
✅ 다운로드 완료: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/184.mp3

[3/185] Processing: https://www.fss.or.kr/fss/bbs/B0000206/view.do?nttId=36742&menuNo=200690&pageIndex=1
title >>  KB저축은행인데요 카카오톡친구추천부탁드립니다
등록일 >> 2020-12-10
✅ 다운로드 완료: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/183.mp3

[4/185] Processing: https://www.fss.or.kr/fss/bbs/B0000206/view.do?nttId=36741&menuNo=200690&pageIndex=1
title >>  영업점에선 절 모를수도있어요(...___)
등록일 >> 2020-12-10
✅ 다운로드 완료: 

[4] 대출사기형 mp3 정보 데이터 csv 파일로 저장

In [ ]:
import csv
import os

# CSV 파일 저장 경로 설정 (파일명 변경)
csv_file_path = os.path.join(download_dir_2, "대출사기형mp3_정보.csv")

# CSV 파일로 저장
with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
    # 필드명 설정
    fieldnames = ['post_number', 'title', 'date', 'url', 'filename', 'path']

    # CSV 작성자 생성
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 헤더 작성
    writer.writeheader()

    # 데이터 작성
    for item in collected_mp3_data:
        writer.writerow(item)

print(f"✅ CSV 파일 저장 완료: {csv_file_path}")

✅ CSV 파일 저장 완료: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/대출사기형/대출사기형mp3_정보.csv


#<금융감독원 '그놈 목소리' 수사기관 사칭형 크롤링>

In [ ]:
import re                                     # 정규표현식 사용을 위한 모듈
import requests                               # 웹 요청을 위한 모듈
from bs4 import BeautifulSoup                 # HTML 파싱을 위한 모듈
from urllib.parse import urljoin, unquote     # 상대 URL을 절대 URL로 바꿔주는 함수

# 기본 url 설정
base_url   = "https://www.fss.or.kr"

# 첫 번째 페이지 URL (전체 페이지 수를 가져오기 위해 필요)
first_page_url_3 = "https://www.fss.or.kr/fss/bbs/B0000207/list.do?menuNo=200691"

# 첫 페이지 요청
res = requests.get(first_page_url_3)
soup = BeautifulSoup(res.text, "html.parser")

[1] 전체 페이지 수 수집

In [ ]:
# 전체 페이지 수가 들어 있는 div 선택
count_total_div = soup.select_one("div.count-total")

# 해당 div의 전체 텍스트 가져오기
text = count_total_div.get_text(strip=True)

# 정규표현식으로 "/19페이지"에서 숫자 19만 추출
match = re.search(r"/\s*(\d+)\s*페이지", text)

# 추출된 숫자를 정수형으로 변환하여 total_pages에 저장 (매칭 안 되면 기본값 1)
total_pages_3 = int(match.group(1)) if match else 1

print(f"📄 전체 페이지 수: {total_pages_3}")

📄 전체 페이지 수: 23


[2] 1페이지 ~ 전체 페이지 순환하며 상세 페이지 수집

In [ ]:
# 페이지별 url 템플릿
page_template_3 = "https://www.fss.or.kr/fss/bbs/B0000207/list.do?menuNo=200691&pageIndex={}"

# 상세 페이지 URL들을 저장할 리스트
all_detail_urls_3 = []

# 각 페이지를 순회하며 링크 추출
for page in range(1, total_pages_3 + 1):
    url = page_template_3.format(page)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    # 각 게시글 행을 순회
    for tr in soup.select("tbody tr"):
        num_td = tr.select_one("td.num")
        print("num_td >> ", num_td)
        title_a = tr.select_one("td.title a")
        print("titlte_a >> ", title_a)

        if num_td and title_a:
            post_number = num_td.get_text(strip=True)
            print("post_number >> ", post_number)
            href = title_a["href"]
            print("href >> ", href)
            full_url = urljoin(base_url, href)
            print("full_url >> ", full_url)

            all_detail_urls_3.append((post_number, full_url))

# 수집된 상세페이지 수 출력
print(f"🔎 수집된 상세페이지 수: {len(all_detail_urls_3)}")

# 예시 출력 (앞부분만)
for num, url in all_detail_urls_3[:5]:
    print(f"📌 글번호 {num} → {url}")

num_td >>  <td class="num"> 
					227
					</td>
titlte_a >>  <a href="/fss/bbs/B0000207/view.do?nttId=36745&amp;menuNo=200691&amp;pageIndex=1">금융범죄수사중입니다. 사건내용을 유출할 경우 처벌받을수있습니다</a>
post_number >>  227
href >>  /fss/bbs/B0000207/view.do?nttId=36745&menuNo=200691&pageIndex=1
full_url >>  https://www.fss.or.kr/fss/bbs/B0000207/view.do?nttId=36745&menuNo=200691&pageIndex=1
num_td >>  <td class="num"> 
					226
					</td>
titlte_a >>  <a href="/fss/bbs/B0000207/view.do?nttId=36740&amp;menuNo=200691&amp;pageIndex=1">본인과 연루된 불법 명의도용사건이 검찰에 접수되어 연락드렸습니다</a>
post_number >>  226
href >>  /fss/bbs/B0000207/view.do?nttId=36740&menuNo=200691&pageIndex=1
full_url >>  https://www.fss.or.kr/fss/bbs/B0000207/view.do?nttId=36740&menuNo=200691&pageIndex=1
num_td >>  <td class="num"> 
					225
					</td>
titlte_a >>  <a href="/fss/bbs/B0000207/view.do?nttId=36739&amp;menuNo=200691&amp;pageIndex=1">다음주에 재판이 열립니다. 재판증거자료를 위해 통화를 녹취하겠습니다.</a>
post_number >>  225
href >>  /fss/bbs/B0000207/view.do?nttId=367

[3] 전체 페이지 순회하며 모든 페이지의 mp3 파일 다운로드

In [ ]:
import os

# 저장 경로 생성
download_dir_3 = "/content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/수사기관_사칭형"
os.makedirs(download_dir_3, exist_ok=True)

# 다운로드 결과를 저장할 리스트
collected_mp3_data_2 = []

In [ ]:
# 파일명에서 특수문자 제거 및 길이 제한
def sanitize_filename(title, max_length=100):
    title = re.sub(r'[\\/*?:"<>|]', "_", title)  # 파일명에 쓸 수 없는 문자 제거
    return title.strip()[:max_length]

# 전체 상세 페이지 순회
for idx, (post_number, detail_url) in enumerate(all_detail_urls_3):
    print(f"\n[{idx+1}/{len(all_detail_urls_3)}] Processing: {detail_url}")

    try:
        # 상세 페이지 요청
        res = requests.get(detail_url)
        soup = BeautifulSoup(res.text, "html.parser")

        # 제목 추출
        title_tag = soup.select_one("h2.subject")
        if not title_tag:
            print("❌ 제목 태그 없음. 스킵.")
            continue
        title_text = title_tag.get_text(strip=True)
        safe_title = sanitize_filename(title_text)
        print("title >> ", safe_title)

        # ✅ 등록일 추출 (dt, dd 쌍을 순회하면서 "등록일" 텍스트를 가진 dt를 찾음)
        post_date = "unknown"
        bd_tag = soup.select_one("div.bd-view")
        if bd_tag:
            dd_tags = bd_tag.select("dd")
            dd_tag  = dd_tags[1]
            post_date = dd_tag.get_text().strip()

        print("등록일 >>", post_date)


        # video 태그 선택
        video_tag = soup.select_one("div.dbdata video")
        if not video_tag or not video_tag.has_attr("src"):
            print("⚠️ video 태그 또는 src 없음. 스킵.")
            continue

        # src에서 상대경로 추출
        relative_url = video_tag["src"].replace("&amp;", "&")
        download_url = urljoin("https://www.fss.or.kr", relative_url)

        # 파일명 생성 (글번호_제목 형식)
        filename = f"{post_number}.mp3"  # 글 번호만 사용
        file_path = os.path.join(download_dir_3, filename)

        # 파일 다운로드
        response = requests.get(download_url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024*1024):  # 1MB 단위
                    if chunk:
                        f.write(chunk)
            print(f"✅ 다운로드 완료: {file_path}")

            collected_mp3_data_2.append({
                "post_number": post_number,
                "title": title_text,
                "date": post_date,
                "url": detail_url,
                "filename": filename,
                "path": file_path
            })
        else:
            print(f"❌ 다운로드 실패. 상태코드: {response.status_code}")

    except Exception as e:
        print(f"❌ 예외 발생: {e}")

print("\n📦 모든 mp3 다운로드 작업 완료.")


[1/227] Processing: https://www.fss.or.kr/fss/bbs/B0000207/view.do?nttId=36745&menuNo=200691&pageIndex=1
title >>  금융범죄수사중입니다. 사건내용을 유출할 경우 처벌받을수있습니다
등록일 >> 2020-12-10
✅ 다운로드 완료: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/수사기관_사칭형/227.mp3

[2/227] Processing: https://www.fss.or.kr/fss/bbs/B0000207/view.do?nttId=36740&menuNo=200691&pageIndex=1
title >>  본인과 연루된 불법 명의도용사건이 검찰에 접수되어 연락드렸습니다
등록일 >> 2020-12-10
✅ 다운로드 완료: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/수사기관_사칭형/226.mp3

[3/227] Processing: https://www.fss.or.kr/fss/bbs/B0000207/view.do?nttId=36739&menuNo=200691&pageIndex=1
title >>  다음주에 재판이 열립니다. 재판증거자료를 위해 통화를 녹취하겠습니다.
등록일 >> 2020-12-10
✅ 다운로드 완료: /content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/수사기관_사칭형/225.mp3

[4/227] Processing: https://www.fss.or.kr/fss/bbs/B0000207/view.do?nttId=36736&menuNo=200691&pageIndex=1
title >>  강남서 사이버수사과입니

[4] 대출사기형 mp3 정보 데이터 csv 파일로 저장

In [ ]:
import csv
import os

# CSV 파일 저장 경로 설정
csv_file_path_2 = os.path.join(download_dir_3, "수사기관사칭형mp3_정보.csv")

# CSV 파일로 저장
with open(csv_file_path_2, 'w', newline='', encoding='utf-8-sig') as csvfile:
    # 필드명 설정
    fieldnames = ['post_number', 'title', 'date', 'url', 'filename', 'path']

    # CSV 작성자 생성
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 헤더 작성
    writer.writeheader()

    # 데이터 작성
    for item in collected_mp3_data_2:
        writer.writerow(item)

print(f"✅ CSV 파일 저장 완료: {csv_file_path_2}")

In [ ]:
import csv
import os
import pandas as pd

download_dir_3 = "/content/drive/MyDrive/2025_VoicePhshing_Detection_Model/dataset/FSS_voicephishing_data/mp3/수사기관_사칭형"

csv_file_path_2 = os.path.join(download_dir_3, "수사기관사칭형mp3_정보.csv")

# CSV를 DataFrame으로 불러오고 상위 10줄 출력
df = pd.read_csv(csv_file_path_2, encoding='utf-8')  # 문제가 생기면 'utf-8-sig'로 변경
df.head(10)

,post_number,title,date,url,filename,path
0,227,금융범죄수사중입니다. 사건내용을 유출할 경우 처벌받을수있습니다,2020-12-10,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,227.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
1,226,본인과 연루된 불법 명의도용사건이 검찰에 접수되어 연락드렸습니다,2020-12-10,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,226.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
2,225,다음주에 재판이 열립니다. 재판증거자료를 위해 통화를 녹취하겠습니다.,2020-12-10,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,225.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
3,224,강남서 사이버수사과입니다,2020-12-07,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,224.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
4,223,"이 사건은 전자금융거래법 위반 금융사기 사건으로, 다수의 피해자가 발생한 고소고발 ...",2018-06-14,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,223.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
5,222,이 전화는 녹취전화라 수신이 안되므로 혹시 향후 궁금한 점 생기시면 서울중앙지검에서...,2018-06-14,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,222.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
6,221,(동일 사기범 다수 제보 접수 건 편집본) 저는 서울중앙지검 강력수사팀 OOO 수사...,2018-06-12,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,221.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
7,220,"(동일 사기범 다수 제보 건 편집본) 녹취라고 해서 별다른 건 없고, 아까 제가 드...",2018-06-12,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,220.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
8,219,(동일 사기범 다수 제보 건 편집본) 귀하의 이름을 판매자로 등록한 후 고가의 항공...,2018-06-12,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,219.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
9,218,(동일 사기범 다수 제보 건 편집본) 중고나라 사이트를 통해 고가의 물건을 등록해놓...,2018-06-12,https://www.fss.or.kr/fss/bbs/B0000207/view.do...,218.mp3,/content/drive/MyDrive/2025_VoicePhshing_Detec...
